In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/Shareddrives/xm/torchkge

In [ ]:
!python3 test.py

In [ ]:
!python3 test.py

In [ ]:
!python3 test.py

In [ ]:
!python3 test.py

In [4]:
!git clone https://github.com/Moon-xm/torchkge.git

Cloning into 'torchkge'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 3458 (delta 71), reused 101 (delta 46), pack-reused 3321
Receiving objects: 100% (3458/3458), 64.76 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (2273/2273), done.
Checking out files: 100% (91/91), done.


In [5]:
cd torchkge

/kaggle/working/torchkge


In [6]:
!ls

GADM9_TransD.py		GeoDBpedia21_TransR.py	docs		      tests
GADM9_TransE.py		LICENSE			requirements_dev.txt  torchkge
GADM9_TransE_GDR.py	README.rst		setup.cfg	      tox.ini
GADM9_TransR.py		benchmarks		setup.py
GeoDBpedia21_TransD.py	checkpoint		test_ignite.py
GeoDBpedia21_TransE.py	dataprocess.py		test_short.py


In [7]:
!python3 GADM9_TransE.py

Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
loading ckpt sucessful...
TransEModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
)
lr: 0.0001, margin: 1.5, dim 100, total epoch: 2000, device: cuda:0, batch size: 5120,optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 1e-05
)
Training ...
Epoch [1200/2000] | mean loss:   50.483, time: 0:00:43
Train loss:   50.483, Val Hit@10: 70.75%, Time 0:01:24 
Epoch [1267/2000] | mean loss:   49.089, time: 0:01:53^C
Traceback (most recent call last):
  File "GADM9_TransE.py", line 135, in <module>
    main()
  File "GADM9_TransE.py", line 95, in main
    loss.backward()
  File "/opt/conda/lib/python3.7/site-packages/torch/tensor.py", line 221, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/opt/conda/lib/python3.7/sit

GADM9_TransE_Adam.py

In [8]:
from torch import cuda
from torch.optim import Adam, SGD
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransE'
    benchmarks = 'GADM9'

    emb_dim = 100
    lr = 0.0001
    margin = 1.5

    n_epochs = 5000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 512  # 测评valid test 时batch size
    validation_freq = 200 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = 500  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
#     optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    optimiezer = SGD(model.parameters(), lr=lr)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {},optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10(filter): {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("\nNo optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
loading ckpt sucessful, start on epoch 1101...
TransEModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
)
lr: 0.0001, margin: 1.5, dim 100, total epoch: 5000, device: cuda:0, batch size: 5120,optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 1e-05
)
Training ...
Epoch [1200/5000] | mean loss:   50.586, time: 0:00:43
Train loss:   50.586, Val Hit@10: 70.87%, Time 0:01:24 
Epoch [1400/5000] | mean loss:   46.193, time: 0:02:49
Train loss:   46.193, Val Hit@10: 69.42%, Time 0:03:30 
Epoch [1600/5000] | mean loss:   44.247, time: 0:04:54
Train loss:   44.247, Val Hit@10: 68.68%, Time 0:05:35 
Epoch [1602/5000] | mean loss:   44.138, time: 0:05:36No optimization for a long time, auto-stopping...
Training done, start evaluate on test data...
Hit@10 : 0.606 	

SGD

In [12]:
from torch import cuda
from torch.optim import Adam, SGD
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransE'
    benchmarks = 'GADM9'

    emb_dim = 100
    lr = 0.0001
    margin = 1.5

    n_epochs = 5000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 512  # 测评valid test 时batch size
    validation_freq = 200 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = 1000  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '_SGD.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
#     optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    optimizer = SGD(model.parameters(), lr=lr)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {},optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10(filter): {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("No optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
TransEModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
)
lr: 0.0001, margin: 1.5, dim 100, total epoch: 5000, device: cuda:0, batch size: 5120,optim: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Training ...
Epoch [ 200/5000] | mean loss: 1501.833, time: 0:01:17
Train loss: 1501.833, Val Hit@10(filter): 9.77%, Time 0:01:58 *
Epoch [ 400/5000] | mean loss: 1343.400, time: 0:03:14
Train loss: 1343.400, Val Hit@10(filter): 9.96%, Time 0:03:55 *
Epoch [ 600/5000] | mean loss: 1212.495, time: 0:05:12
Train loss: 1212.495, Val Hit@10(filter): 10.21%, Time 0:05:53 *
Epoch [ 800/5000] | mean loss: 1070.639, time: 0:07:10
Train loss: 1070.639, Val Hit@10(filter): 10.53%, Time 0:07:51 *
Epoch [1000/5000] | mean loss:  950.793, time: 0:09:08
Train loss:  950.793, V

In [18]:
from torch import cuda
from torch.optim import Adam, SGD
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransE'
    benchmarks = 'GADM9'

    emb_dim = 100
    lr = 0.0001
    margin = 1.5

    n_epochs = 20000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 512  # 测评valid test 时batch size
    validation_freq = 500 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = 2000  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '_SGD.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
#     optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    optimizer = SGD(model.parameters(), lr=lr)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {},optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10(filter): {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("No optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
loading ckpt sucessful, start on epoch 5001...
TransEModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
)
lr: 0.0001, margin: 1.5, dim 100, total epoch: 20000, device: cuda:0, batch size: 5120,optim: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Training ...
Epoch [5500/20000] | mean loss:  118.078, time: 0:03:12
Train loss:  118.078, Val Hit@10(filter): 52.42%, Time 0:03:53 *
Epoch [6000/20000] | mean loss:  107.632, time: 0:07:05
Train loss:  107.632, Val Hit@10(filter): 53.82%, Time 0:07:46 *
Epoch [6500/20000] | mean loss:  100.063, time: 0:10:58
Train loss:  100.063, Val Hit@10(filter): 54.80%, Time 0:11:39 *
Epoch [7000/20000] | mean loss:   93.712, time: 0:14:52
Train loss:   93.712, Val Hit@10(filter): 55.55%, Time 0:15:33 *
Epoch [7500/20000] | mea

In [19]:
from torch import cuda
from torch.optim import Adam, SGD
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransE'
    benchmarks = 'GADM9'

    emb_dim = 100
    lr = 0.0001
    margin = 1.5

    n_epochs = 50000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 512  # 测评valid test 时batch size
    validation_freq = 500 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = 2000  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '_SGD.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
#     optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    optimizer = SGD(model.parameters(), lr=lr)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {},optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10(filter): {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("No optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
loading ckpt sucessful, start on epoch 20001...
TransEModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
)
lr: 0.0001, margin: 1.5, dim 100, total epoch: 50000, device: cuda:0, batch size: 5120,optim: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Training ...
Epoch [20500/50000] | mean loss:   44.903, time: 0:03:12
Train loss:   44.903, Val Hit@10(filter): 63.47%, Time 0:03:53 *
Epoch [21000/50000] | mean loss:   45.071, time: 0:07:06
Train loss:   45.071, Val Hit@10(filter): 63.71%, Time 0:07:47 *
Epoch [21500/50000] | mean loss:   43.949, time: 0:11:00
Train loss:   43.949, Val Hit@10(filter): 63.77%, Time 0:11:40 *
Epoch [22000/50000] | mean loss:   44.247, time: 0:14:53
Train loss:   44.247, Val Hit@10(filter): 64.00%, Time 0:15:34 *
Epoch [22500/50000]

In [20]:
cd ..

/kaggle/working


In [22]:
!mkdir torchkge2

In [23]:
cd torchkge2

/kaggle/working/torchkge2


In [25]:
!git clone https://github.com/Moon-xm/torchkge.git --branch Geo

Cloning into 'torchkge'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 3502 (delta 98), reused 138 (delta 65), pack-reused 3321
Receiving objects: 100% (3502/3502), 74.59 MiB | 24.85 MiB/s, done.
Resolving deltas: 100% (2300/2300), done.
Checking out files: 100% (94/94), done.


In [26]:
cd torchkge

/kaggle/working/torchkge2/torchkge


### GeoDBpedia21(new)


In [31]:
from torch import cuda
from torch.optim import Adam, SGD
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransE'
    benchmarks = 'GeoDBpedia21'

    emb_dim = 100
    lr = 0.001
    margin = 10

    n_epochs = 20000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 128  # 测评valid test 时batch size
    validation_freq = 500  # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = 2000  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks + '_' + model_name + '.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {},optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10(filter): {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("\nNo optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 84988 entities, 21 relations, 91217 triplets.
Valid set: 3973 triplets, Test set: 3982 triplets.
Loading model...
loading ckpt sucessful, start on epoch 21...
TransEModel(
  (ent_emb): Embedding(84988, 100)
  (rel_emb): Embedding(21, 100)
)
lr: 0.001, margin: 10, dim 100, total epoch: 20000, device: cuda:0, batch size: 5120,optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 1e-05
)
Training ...
Epoch [ 500/20000] | mean loss: 2350.876, time: 0:01:013
Train loss: 2350.876, Val Hit@10: 7.92%, Time 0:01:07 *
Epoch [1000/20000] | mean loss: 2304.272, time: 0:02:10
Train loss: 2304.272, Val Hit@10: 7.90%, Time 0:02:16 
Epoch [1500/20000] | mean loss: 2347.319, time: 0:03:19
Train loss: 2347.319, Val Hit@10: 7.79%, Time 0:03:24 
Epoch [2000/20000] | mean loss: 2246.656, time: 0:04:28
Train loss: 2246.656, Val Hit@10: 8.23%, Time 0:04:34 *
Epoch [2500/20000] | mean loss: 2280.063, time: 0:05:37
T

In [33]:
from torch import cuda
from torch.optim import Adam, SGD
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransE'
    benchmarks = 'GeoDBpedia21'

    emb_dim = 100
    lr = 0.001
    margin = 10

    n_epochs = 20000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 128  # 测评valid test 时batch size
    validation_freq = 500  # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = 2000  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks + '_' + model_name + '_SGD.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    optimizer = SGD(model.parameters(), lr=lr)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
#     scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {},optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
#         scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10(filter): {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("\nNo optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 84988 entities, 21 relations, 91217 triplets.
Valid set: 3973 triplets, Test set: 3982 triplets.
Loading model...
TransEModel(
  (ent_emb): Embedding(84988, 100)
  (rel_emb): Embedding(21, 100)
)
lr: 0.001, margin: 10, dim 100, total epoch: 20000, device: cuda:0, batch size: 5120,optim: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)
Training ...
Epoch [ 500/20000] | mean loss: 2398.119, time: 0:00:522
Train loss: 2398.119, Val Hit@10(filter): 9.25%, Time 0:00:58 *
Epoch [1000/20000] | mean loss: 2185.000, time: 0:01:49
Train loss: 2185.000, Val Hit@10(filter): 8.21%, Time 0:01:55 
Epoch [1500/20000] | mean loss: 2063.176, time: 0:02:46
Train loss: 2063.176, Val Hit@10(filter): 8.83%, Time 0:02:52 
Epoch [2000/20000] | mean loss: 2096.423, time: 0:03:43
Train loss: 2096.423, Val Hit@10(filter): 8.44%, Time 0:03:49 
Epoch [2500/20000] | mean loss: 2014.447, time: 0:04:40
Train loss: 2014.447, Val

### GADM9

#### TransR

In [ ]:
from torch import cuda
from torch.optim import Adam
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransR'
    benchmarks = 'GADM9'

    emb_dim = 100
    ent_dim = emb_dim
    rel_dim = emb_dim
    lr = 0.0001
    margin = 1

    n_epochs = 20000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 128  # 测评valid test 时batch size
    validation_freq = 200 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = validation_freq*5  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '_Adam.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(ent_dim, rel_dim, kg_train.n_ent, kg_train.n_rel)
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {}, optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10: {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("\nNo optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
loading ckpt sucessful, start on epoch 21...
TransRModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
  (proj_mat): Embedding(9, 10000)
)
lr: 0.0001, margin: 1, dim 100, total epoch: 20000, device: cuda:0, batch size: 5120, optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 1e-05
)
Training ...
Epoch [ 200/20000] | mean loss:  345.520, time: 0:04:24

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:  345.520, Val Hit@10: 9.34%, Time 0:05:16 *
Epoch [ 400/20000] | mean loss:   70.681, time: 0:10:10

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:   70.681, Val Hit@10: 14.42%, Time 0:11:02 *
Epoch [ 600/20000] | mean loss:   30.290, time: 0:15:56

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:   30.290, Val Hit@10: 19.28%, Time 0:16:47 *
Epoch [ 800/20000] | mean loss:   16.846, time: 0:21:42

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:   16.846, Val Hit@10: 24.14%, Time 0:22:34 *
Epoch [1000/20000] | mean loss:   11.951, time: 0:27:28

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]

In [2]:
cd torchkge

/kaggle/working/torchkge


In [3]:
from torch import cuda
from torch.optim import Adam
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransR'
    benchmarks = 'GADM9'

    emb_dim = 100
    ent_dim = emb_dim
    rel_dim = emb_dim
    lr = 0.0001
    margin = 1

    n_epochs = 20000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 128  # 测评valid test 时batch size
    validation_freq = 200 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = validation_freq*5  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '_Adam.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(ent_dim, rel_dim, kg_train.n_ent, kg_train.n_rel)
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {}, optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10: {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("\nNo optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
TransRModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
  (proj_mat): Embedding(9, 10000)
)
lr: 0.0001, margin: 1, dim 100, total epoch: 20000, device: cuda:0, batch size: 5120, optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 1e-05
)
Training ...
Epoch [ 200/20000] | mean loss:  382.378, time: 0:05:23

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:  382.378, Val Hit@10: 9.17%, Time 0:06:26 *


/opt/conda/lib/python3.7/site-packages/torch/optim/optimizer.py:184: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  if p.grad is not None:
/opt/conda/lib/python3.7/site-packages/torch/optim/adam.py:78: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  if p.grad is not 

Epoch [ 400/20000] | mean loss:   64.926, time: 0:11:50

...

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:    3.057, Val Hit@10: 48.35%, Time 2:34:03 *
Epoch [5000/20000] | mean loss:    3.089, time: 2:39:26

Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]


Train loss:    3.089, Val Hit@10: 48.70%, Time 2:40:30 *
Epoch [5003/20000] | mean loss:    3.908, time: 2:40:35

KeyboardInterrupt: 

In [4]:
from torch import cuda
from torch.optim import Adam
import torch

# from torchkge.models import TransEModel
from torchkge.sampling import BernoulliNegativeSampler
from torchkge.utils import MarginLoss, DataLoader
# from torchkge.utils.datasets import load_GADM9
from torchkge.evaluation import LinkPredictionEvaluator

from torchkge.utils.my_utils import load_ckpt, save_ckpt, create_dir_not_exists,time_since

import os
from importlib import import_module
from tqdm.autonotebook import tqdm
import time


def main():
    # Define some hyper-parameters for training
    model_name = 'TransR'
    benchmarks = 'GADM9'

    emb_dim = 100
    ent_dim = emb_dim
    rel_dim = emb_dim
    lr = 0.0001
    margin = 1

    n_epochs = 5000
    train_b_size = 5120  # 训练时batch size
    eval_b_size = 128  # 测评valid test 时batch size
    validation_freq = 200 # 多少轮进行在验证集进行一次测试 同时保存最佳模型
    require_improvement = validation_freq*5  # 验证集top_k超过多少epoch没下降，结束训练
    model_save_path = './checkpoint/' + benchmarks +'_'+ model_name + '_Adam.ckpt'  # 保存最佳hits k (ent)模型
    device = 'cuda:0' if cuda.is_available() else 'cpu'

    # Load dataset
    module = getattr(import_module('torchkge.models'), model_name+'Model')
    load_data = getattr(import_module('torchkge.utils.datasets'), 'load_'+benchmarks)

    print('Loading data...')
    kg_train, kg_val, kg_test = load_data()
    print(f'Train set: {kg_train.n_ent} entities, {kg_train.n_rel} relations, {kg_train.n_facts} triplets.')
    print(f'Valid set: {kg_val.n_facts} triplets, Test set: {kg_test.n_facts} triplets.')

    # Define the model and criterion
    print('Loading model...')
    model = module(ent_dim, rel_dim, kg_train.n_ent, kg_train.n_rel)
    # model = TransEModel(emb_dim, kg_train.n_ent, kg_train.n_rel, dissimilarity_type='L2')
    criterion = MarginLoss(margin)
    # Move everything to CUDA if available
    if device == 'cuda:0':
        cuda.empty_cache()
        model.to(device)
        criterion.to(device)
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda='all')
    else:
        dataloader = DataLoader(kg_train, batch_size=train_b_size, use_cuda=None)


    # Define the torch optimizer to be used
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

    sampler = BernoulliNegativeSampler(kg_train)

    start_epoch = 1
    best_score = float('-inf')

    if os.path.exists(model_save_path):  # 存在则加载模型 并继续训练
        start_epoch, best_score = load_ckpt(model_save_path, model, optimizer)
        print(f'loading ckpt sucessful, start on epoch {start_epoch}...')
    print(model)
    print('lr: {}, margin: {}, dim {}, total epoch: {}, device: {}, batch size: {}, optim: {}'\
    .format(lr, margin, emb_dim, n_epochs, device, train_b_size, optimizer))

    # iterator = tqdm(range(start_epoch, n_epochs+1), unit='epoch')
    print('Training ...')

    last_improve = start_epoch  # 记录上次验证集loss下降的epoch数

    start = time.time()
    for epoch in range(start_epoch, n_epochs+1):
        # scheduler.step()  # lr衰减
        running_loss = 0.0
        model.train()
        for i, batch in enumerate(dataloader):
            h, t, r = batch[0], batch[1], batch[2]
            n_h, n_t = sampler.corrupt_batch(h, t, r)
            optimizer.zero_grad()

            # forward + backward + optimize
            pos, neg = model(h, t, n_h, n_t, r)
            loss = criterion(pos, neg)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print('\rEpoch [{:>4}/{:>4}] | mean loss: {:>8.3f}, time: {}'.format(epoch, n_epochs, running_loss / len(dataloader), time_since(start)), end='',flush=True)
        # iterator.set_description(
        #     'Epoch {:>5} | mean loss: {:>8.3f}'.format(epoch,
        #                                           running_loss / len(dataloader)))
        # test
        if epoch % validation_freq == 0:
            create_dir_not_exists('./checkpoint')
            model.eval()
            evaluator = LinkPredictionEvaluator(model, kg_val)
            evaluator.evaluate(b_size=eval_b_size, verbose=False)
            _, hit_at_k = evaluator.hit_at_k(10)  # val filter hit_k
            if hit_at_k > best_score:
                save_ckpt(model, optimizer, epoch, best_score, model_save_path)
                best_score = hit_at_k
                improve = '*'  # 在有提升的结果后面加上*标注
                last_improve = epoch  # 验证集hit_k增大即认为有提升
            else:
                improve = ''
            msg = '\nTrain loss: {:>8.3f}, Val Hit@10: {:>5.2%}, Time {} {}'
            print(msg.format(running_loss / len(dataloader), hit_at_k, time_since(start), improve))
        model.normalize_parameters()
        if epoch - last_improve > require_improvement:
            # 验证集loss超过1000batch没下降，结束训练
            print("\nNo optimization for a long time, auto-stopping...")
            break

    print('Training done, start evaluate on test data...')
    # Testing the best checkpoint on test dataset
    load_ckpt(model_save_path, model, optimizer)
    model.eval()
    evaluator = LinkPredictionEvaluator(model, kg_test)
    evaluator.evaluate(eval_b_size, verbose=False)
    evaluator.print_results()


if __name__ == "__main__":
    main()


Loading data...
Train set: 34715 entities, 9 relations, 398758 triplets.
Valid set: 49845 triplets, Test set: 49849 triplets.
Loading model...
loading ckpt sucessful, start on epoch 5001...
TransRModel(
  (ent_emb): Embedding(34715, 100)
  (rel_emb): Embedding(9, 100)
  (proj_mat): Embedding(9, 10000)
)
lr: 0.0001, margin: 1, dim 100, total epoch: 5000, device: cuda:0, batch size: 5120, optim: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 1e-05
)
Training ...
Training done, start evaluate on test data...


Projecting entities:   0%|          | 0/34715 [00:00<?, ?entities/s]

Hit@10 : 0.419 		 Filt. Hit@10 : 0.485
Mean Rank : 512 	 Filt. Mean Rank : 314
MRR : 0.164 		 Filt. MRR : 0.205


In [1]:
!git clone https://github.com/Moon-xm/torchkge.git

Cloning into 'torchkge'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 3522 (delta 114), reused 149 (delta 71), pack-reused 3321
Receiving objects: 100% (3522/3522), 74.59 MiB | 13.24 MiB/s, done.
Resolving deltas: 100% (2316/2316), done.
